# LP Duality and Revenue Management

## Question 1: Semiconductor Manufacturing

### Part 1: LP Formulation

First, let's load Gurobi.

In [1]:
from gurobipy import *

Now we construct the model, variables, constraints and objective function.

In [2]:
# Create the model. 
mod_1 = Model()

# Add the variables. 
x_1 = mod_1.addVar()
x_2 = mod_1.addVar()
x_3 = mod_1.addVar()

# Resource constraints:
ge_constr = mod_1.addConstr(14*x_1 + 20*x_2 + 40*x_3 <= 300)
si_constr = mod_1.addConstr(30*x_1 + 20*x_2 + 15*x_3 <= 200)
time_constr = mod_1.addConstr((1/3)*x_1 + (1/2)*x_2 + (5/6)*x_3 <= 18)

# Nonnegativity constraints:
mod_1.addConstr(x_1 >= 0.0)
mod_1.addConstr(x_2 >= 0.0)
mod_1.addConstr(x_3 >= 0.0)

# Create the objective function.
mod_1.setObjective(500*x_1 + 800*x_2 + 1000*x_3, GRB.MAXIMIZE)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-31


Now let's update and solve our model.

In [3]:
mod_1.update()

mod_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 3 columns and 12 nonzeros
Model fingerprint: 0xd0f5e2c1
Coefficient statistics:
  Matrix range     [3e-01, 4e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2000000e+04   8.114000e+00   0.000000e+00      0s
       2    9.6000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.600000000e+03


Now let's obtain our optimal values below.

In [7]:
# Extract the solution status. 
status = mod_1.status
print(status)
if status == GRB.OPTIMAL:
    print("Solved to optimality")
    
# Extract the optimal values of the decision variables.
chip_1 = x_1.x
chip_2 = x_2.x
chip_3 = x_3.x

print("Chip 1 Production: ", chip_1)
print("Chip 2 Production: ", chip_2)
print("Chip 3 Production: ", chip_3)

# Extract the optimal objective value.
optimal_obj = mod_1.objval
print("Optimal objective: ", optimal_obj)

2
Solved to optimality
Chip 1 Production:  0.0
Chip 2 Production:  7.0
Chip 3 Production:  4.0
Optimal objective:  9600.0


### Part 2: Dual Formulation

Let's solve the dual problem below.

In [8]:
m_dual = Model()

p_G = m_dual.addVar()
p_S = m_dual.addVar()
p_T = m_dual.addVar()

# Create the dual constraints
m_dual.addConstr(14*p_G + 30*p_S + (1/3)*p_T >= 500)
m_dual.addConstr(20*p_G + 20*p_S + (1/2)*p_T >= 800)
m_dual.addConstr(40*p_G + 15*p_S + (5/6)*p_T >= 1000)

m_dual.setObjective(300*p_G + 200*p_S + 18*p_T, GRB.MINIMIZE)

m_dual.update()

m_dual.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xf874cc4a
Coefficient statistics:
  Matrix range     [3e-01, 4e+01]
  Objective range  [2e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.875000e+02   0.000000e+00      0s
       2    9.6000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.600000000e+03


In [9]:
p_G_value = p_G.x
p_S_value = p_S.x
p_T_value = p_T.x

print("Dual variable for Germanium: ", p_G_value)
print("Dual variable for Silicon: ", p_S_value)
print("Dual variable for Time: ", p_T_value)

# Extract the dual optimal objective value.
dual_optimal_obj = m_dual.objval
print("Dual optimal objective: ", dual_optimal_obj)

Dual variable for Germanium:  16.0
Dual variable for Silicon:  24.0
Dual variable for Time:  0.0
Dual optimal objective:  9600.0


Let's verify the optimal values are the same as the shadow prices of the constraints.

In [10]:
dual_values = [ge_constr.pi, si_constr.pi]
print(dual_values)

[16.0, 24.0]


### Part 3: Shadow Prices / Marginal Analysis

Let's see what happens when we increase the amount of available Germanium transistors by 10 billion.

In [11]:
ge_constr.rhs = 310

mod_1.update()

mod_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 3 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 4e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.7600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  9.760000000e+03


Now let's see what happens when we increase the amount of available Germanium transistors by 300 billion.

In [12]:
ge_constr.rhs = 600

mod_1.update()

mod_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 3 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 4e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 6e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+04   1.600000e+01   0.000000e+00      0s
       1    1.3333333e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.333333333e+04


Since we did not achieve expected maximum revenue when increasing Germanium units by 300, let's see how much of the resource we are not using when `ge_constr` = 600.

In [13]:
# Access the constraint slacks of the optimal solution.
slack_values = [ge_constr.slack, si_constr.slack, time_constr.slack]
print(slack_values)

[66.66666666666674, 0.0, 6.8888888888888875]


Now let's see what happens when we increase the amount of available Germanium transistors by 10 billion and also increase the amount of available Silicon transistors by 20 billion.

In [14]:
ge_constr.rhs = 310
si_constr.rhs = 220

mod_1.update()

mod_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 6 rows, 3 columns and 12 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 4e+01]
  Objective range  [5e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4666667e+04   3.458333e+01   0.000000e+00      0s
       1    1.0240000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.024000000e+04


## Question 2: Cloud Computing

### Part 2: Solving the capacity control problem in Python/Gurobi

First, create the data.

In [15]:
import numpy as np

# The number of instances
nInstances = 9

# Price for each instance type:
price = np.array([7, 12, 24, 22, 44, 88, 30, 90, 120])

# Rate of each instance type: 
probability = np.array([5.0, 5.0, 1.8, 3.0, 2.6, 1.0, 0.8, 0.4, 0.3])

# Available components per day: 
c = np.array([16, 32, 64, 8, 16, 32, 16, 32, 64])
m = np.array([8, 16, 32, 32, 64, 128, 16, 32, 64])
g = np.array([1, 1, 1, 1, 1, 1, 2, 6, 8])
comp = np.array([c, m, g])

# Time horizon:
T = 5

# Finally, let us compute the forecasted demand for each instance type:
forecast = T * probability
print(forecast)

[25.  25.   9.  15.  13.   5.   4.   2.   1.5]


Now we formulate the LP.

In [16]:
# Create the model
mod_2 = Model()

# Create variables.
x = mod_2.addVars(nInstances, lb = 0, ub = forecast)

# Create the component constraints
c_constr = mod_2.addConstr(sum(c[i] * x[i] for i in range(nInstances)) <= 512)
m_constr = mod_2.addConstr(sum(m[i] * x[i] for i in range(nInstances)) <= 1024)
g_constr = mod_2.addConstr(sum(g[i] * x[i] for i in range(nInstances)) <= 64)

# Set the objective.
mod_2.setObjective(sum(price[i] * x[i] for i in range(nInstances)), GRB.MAXIMIZE)

# Update + solve:
mod_2.update()
mod_2.optimize()

# Get the objective value
LP_obj = mod_2.objval

# Get the allocation
allocation = np.array([x[i].x for i in range(nInstances)])

# Display the results:
print("Allocation, forecast, objective:")
print(allocation)
print(forecast)
print(LP_obj)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Model fingerprint: 0x207d4a09
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [2e+00, 3e+01]
  RHS range        [6e+01, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 8 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+03   7.200000e+01   0.000000e+00      0s
       3    1.0394286e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.039428571e+03
Allocation, forecast, objective:
[6.28571429 0.         0.         3.42857143 0.         5.
 4.         2.         1.5       ]
[25.  25.   9.  15.  13.   5.   4.   2.   1.5]
1039.4285714285716


### Part 3: Simulating current practice

Test out the `generateArrivalSequences` function below.

In [17]:
def generateArrivalSequences(nSimulations, rates, T):
    total_rate = sum(rates)
    nTypes = len(rates)

    arrival_sequences_times = []
    arrival_sequences_types = [];

    for s in range(nSimulations):
        single_arrival_sequence_time = [];
        single_arrival_sequence_type = [];
        t = 0;
        while (t < T):
            single_time = np.random.exponential(1.0/total_rate)
            single_type = np.random.choice(nTypes, p= rates/total_rate )

            t += single_time;

            if (t < T):
                single_arrival_sequence_time.append(t)
                single_arrival_sequence_type.append(single_type)
            else:
                break

        arrival_sequences_times.append(np.array(single_arrival_sequence_time))
        arrival_sequences_types.append(np.array(single_arrival_sequence_type))
    return arrival_sequences_times, arrival_sequences_types

# Code to test out above function
np.random.seed(10)
arrival_sequences_times, arrival_sequences_types = generateArrivalSequences(100, probability, 5)
arrival_sequences_times[0] # Check that we obtained values of 0.07414243, 0.1246028, 0.15928449 for first three times

array([0.07414243, 0.1246028 , 0.15928449, 0.1703762 , 0.17968569,
       0.23779251, 0.23799131, 0.32214315, 0.38642729, 0.51196926,
       0.55126954, 0.57475458, 0.60405595, 0.65238384, 0.70519578,
       0.78740185, 0.90765536, 0.91241996, 0.9185014 , 0.92091505,
       0.96077223, 0.97191941, 0.99369472, 1.01132902, 1.03111801,
       1.05616581, 1.14264621, 1.16700269, 1.38771201, 1.47562187,
       1.52133751, 1.55976805, 1.56178183, 1.56595073, 1.58612921,
       1.58817843, 1.60718495, 1.62855072, 1.73506112, 1.84081623,
       1.88756662, 1.93334033, 1.95132628, 1.96946998, 2.01044946,
       2.04284187, 2.04617896, 2.06703689, 2.25677443, 2.27602698,
       2.3193975 , 2.32294156, 2.39947557, 2.44379428, 2.46549244,
       2.48339641, 2.53963883, 2.54224165, 2.54798842, 2.65703572,
       2.66944199, 2.67512209, 2.70700517, 2.71503145, 2.76708375,
       2.78144725, 2.79437102, 2.92271451, 2.96474668, 2.96549902,
       3.0082648 , 3.04969828, 3.09379581, 3.10187596, 3.16136

Calculate average number of arrivals of type C1 below.

In [18]:
# Count total C1 instances for each sequence
C1_total = 0

for i in range(100):
    C1_count = np.count_nonzero(arrival_sequences_types[i] == 0)
    C1_total += C1_count

# Find average number of arrivals of type C1 
C1_total / len(arrival_sequences_types)

26.63

Calculate average number of arrivals of all types over set of simulated sequences below.

In [19]:
# Count total instances for each sequence
instance_total = 0
arrival_counts = np.array([len(arrival_sequences_types[i]) for i in range(100)])

for i in range(100):
    instance_total += arrival_counts[i]

# Find average number of arrivals for all types
instance_total / len(arrival_sequences_types)

101.0

Implement Cirrus's current policy below.

In [20]:
# Preconditions
nSimulations = 100
nResources = 3
B = np.array([512, 1024, 64])

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy();
    single_revenue = 0.0; # will contain the revenue of this simulation
    nArrivals = len(arrival_sequences_times[s]);

    # Go through the arrivals in sequence
    for j in range(nArrivals):
        # Obtain the time of the arrival, and its type (i)
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is sufficient capacity for the request
        if (c[i] <= b[0]) and (m[i] <= b[1]) and (g[i] <= b[2]):
            # If there is sufficient capacity, accrue the revenue
            # and remove the capacity.
            b[0] -= c[i]
            b[1] -= m[i]
            b[2] -= g[i]
            single_revenue += price[i]

    results_myopic_revenue[s] = single_revenue
    results_myopic_remaining_capacity[0][s] = b[0]
    results_myopic_remaining_capacity[1][s] = b[1]
    results_myopic_remaining_capacity[2][s] = b[2]

# Find the average revenue
mean_myopic_revenue = results_myopic_revenue.mean()
print("Mean revenue (myopic): ", mean_myopic_revenue)
# Find the average remaining quantity of each resource
mean_myopic_remaining_cpu = results_myopic_remaining_capacity[0].mean()
mean_myopic_remaining_mem = results_myopic_remaining_capacity[1].mean()
mean_myopic_remaining_gpu = results_myopic_remaining_capacity[2].mean()
print("Mean CPU remaining (myopic): ", mean_myopic_remaining_cpu)
print("Mean Memory remaining (myopic): ", mean_myopic_remaining_mem)
print("Mean GPU remaining (myopic): ", mean_myopic_remaining_gpu)

Mean revenue (myopic):  528.28
Mean CPU remaining (myopic):  0.24
Mean Memory remaining (myopic):  371.52
Mean GPU remaining (myopic):  37.42


### Part 4: A bid-price control policy

Let's check the shadow prices of the constraints below in order to find our opportunity costs.

In [21]:
dual_values = [c_constr.pi, m_constr.pi, g_constr.pi]
print(dual_values)

[0.10714285714285714, 0.6607142857142857, 0.0]


Next, let's reformulate our LP for a bid-price control policy.

In [25]:
# Preconditions
nSimulations = 100
nResources = 3
B = np.array([512, 1024, 64])
T = 5

instance_types = [ [0],
                   [1],
                   [2],
                   [3],
                   [4],
                   [5],
                   [6],
                   [7],
                   [8] ]

# Create the model
mod_bpc = Model()

# Create variables.
x_bpc = mod_bpc.addVars(nInstances, lb = 0, ub = forecast)

# Create the component constraints
comp_constr = {}
for r in range(nResources):
    comp_constr[r] = mod_bpc.addConstr(sum(comp[r][i]*x_bpc[i] for i in range(nInstances)) <= B[r])

# Set the objective.
mod_bpc.setObjective(sum(price[i] * x_bpc[i] for i in range(nInstances)), GRB.MAXIMIZE)

# Update + solve:
mod_bpc.update()
mod_bpc.optimize()

# Get the objective value
LP_obj_bpc = mod_bpc.objval

# Get the allocation
allocation_bpc = np.array([x_bpc[i].x for i in range(nInstances)])

# Display the results:
print("Allocation, forecast, objective:")
print(allocation_bpc)
print(forecast)
print(LP_obj_bpc)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3 rows, 9 columns and 27 nonzeros
Model fingerprint: 0x207d4a09
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [7e+00, 1e+02]
  Bounds range     [2e+00, 3e+01]
  RHS range        [6e+01, 1e+03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 8 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4400000e+03   7.200000e+01   0.000000e+00      0s
       3    1.0394286e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.039428571e+03
Allocation, forecast, objective:
[6.28571429 0.         0.         3.42857143 0.         5.
 4.         2.         1.5       ]
[25.  25.   9.  15.  13.   5.   4.   2.   1.5]
1039.4285714285716


In [45]:
# As we did in-class, define a function bpc() to re-solve the LO each time:
def bpc(b, t):
    for r in range(nResources):
        comp_constr[r].rhs = b[r]

    for i in range(nInstances):
        x_bpc[i].ub = (T - t)*probability[i]

    # Re-solve the model:
    mod_bpc.update()
    mod_bpc.optimize()
    
    dual_val = [comp_constr[r].pi for r in range(nResources)]

    return dual_val

mod_bpc.Params.outputflag = 0

results_revenue = np.zeros(nSimulations)
results_remaining_capacity = np.zeros((nResources, nSimulations))

for s in range(nSimulations):
    b = B.copy() #Initialize the current capacity
    single_revenue = 0.0 #Initialize the revenue garnered in this simulation
    nArrivals = len(arrival_sequences_times[s])
    for j in range(nArrivals):
        # Take the next arrival time and type from the sequence
        arrival_time = arrival_sequences_times[s][j]
        i = arrival_sequences_types[s][j]

        # Check if there is enough capacity
        if (c[i] <= b[0]) and (m[i] <= b[1]) and (g[i] <= b[2]):
            # Re-solve the LO and obtain the dual values
            dual_val = bpc(b, arrival_time)
            
            total_bid_price = sum([dual_val[r]*comp[r][i] for r in range(nResources)])

            # Check if the revenue is at least the sum of the bid prices:
            if (price[i] >= total_bid_price):
                # If there is sufficient capacity, accrue the revenue
                # and remove the capacity.
                b[0] -= comp[0][i]
                b[1] -= comp[1][i]
                b[2] -= comp[2][i]
                single_revenue += price[i]

    # Save the results of this simulation here:
    results_revenue[s] = single_revenue
    results_remaining_capacity[0][s] = b[0]
    results_remaining_capacity[1][s] = b[1]
    results_remaining_capacity[2][s] = b[2]

# Find the average revenue:
mean_revenue = results_revenue.mean()
print("Mean revenue: ", mean_revenue)
# Find the average remaining quantity of each resource
mean_remaining_cpu = results_remaining_capacity[0].mean()
mean_remaining_mem = results_remaining_capacity[1].mean()
mean_remaining_gpu = results_remaining_capacity[2].mean()
print("Mean CPU remaining: ", mean_remaining_cpu)
print("Mean Memory remaining: ", mean_remaining_mem)
print("Mean GPU remaining: ", mean_remaining_gpu)

Mean revenue:  925.59
Mean CPU remaining:  27.2
Mean Memory remaining:  4.88
Mean GPU remaining:  20.62
